In [4]:
%load_ext autoreload
%autoreload 2
import sys,os
import pickle
from eknot_utils import EventNode,NodeParams
from utils import loadPickle,triHits,norm2,printTopWords,coHits
from eknot_utils import *
from time import time

[resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160107_0114_80_7.pickle')
[Xs,vects,DT,ind2obj] = loadPickle('../data/20160107_0114.pickle')
rootParams,rootNodeDescriptor = loadPickle('../output/plsa_20160107_0114_80.pickle')



In [149]:
topK = 50 # ent used
i = 7
window = 7
t0 = time()
Pw_zs = rootNodeDescriptor.Pw_zs
Pe_z = Pw_zs[1][:,i]
evocab = vects[5].get_feature_names()
vocab = vects[0].get_feature_names()

ent_ind,ents = getEntInd(evocab,Pe_z,topK)

print( "entscore in "+str(time() - t0))
t0 = time()

newsObj = [ind2obj[n] for n in resDocInd]
XN,XEn,NEb,sentencesIn,sentencesInObj,ent_text_n = getNewsContext(newsObj,ent_ind,ents,vocab,window)
print( "get news Context in "+str(time() - t0))
print len(newsObj),len(sentencesIn),len(set(sentencesIn))
t0 = time()

XT,XEt,TEb,tweetsIn,tweetsInObj,ent_text_t = getTweetContext(tweetsObjDedup,ent_ind,ents,vocab,window)
print( "get tweet Context in "+str(time() - t0))



entscore in 0.0428278446198
get news Context in 1.12757515907
28 747 747
append in 2.04305505753
vectorize in 9.5756149292
ents append + vec in 0.996649980545
get tweet Context in 12.6231210232


In [140]:
print np.asarray(NE_.sum(axis=1))[:,0][233]
print sentencesIn[233]
for e in ent_text_n:
    if 'washington' in ent_text_n[e]:
        print e
print type(NE_)    
print min(nScore),nScore[60],nScore[1]
newsScore.argsort()

0.195885078161
elections are always about as "evitable" as one can  imagine.
<class 'scipy.sparse.csr.csr_matrix'>
0.0 0.0 0.0507899195265


array([600, 233,  60, 193, 196,  36,  77, 142, 331, 686, 381,   6, 539,
       597,  72, 143, 543, 270, 525, 104, 704, 391, 698, 509,  16,  33,
       271, 262, 360, 267, 687, 305, 570, 566, 275, 195, 109, 361,   5,
        29, 528, 562, 156, 524, 493, 403, 113,  44,  91,  85, 721, 336,
       643, 688, 319, 744, 740, 648, 596, 693,  31, 741, 397, 459, 186,
       641, 742, 714, 290, 321, 628, 735, 737,   8, 320, 441, 167, 514,
       461, 630, 592, 258, 217, 595, 494, 100, 149, 273, 179, 498, 719,
       623, 117,  87, 518, 692, 644, 412, 639, 676, 165, 706, 631, 169,
       232, 144, 629, 519, 487, 625, 700, 584, 642, 672, 257, 122, 664,
       541,  27, 535, 460, 614, 739, 724, 401, 452, 409,  57, 476, 634,
       324, 718, 261, 399, 322, 652, 425, 712, 572, 194, 627, 587, 713,
       622, 390, 132, 205, 590, 371, 621,  41, 300,  69, 234,  17, 647,
       413, 110, 720, 495, 728,  84, 650, 684, 675, 386, 482, 211, 615,
       537, 710,  15, 492, 422, 190, 438,   3, 396, 667, 417, 26

In [150]:
_XEn = XEn
_XEt = XEt
_NEb = NEb
_TEb = TEb
_XEn.shape

(50, 14118)

In [180]:
"""
#####
rows = []
interestedEnts = ['gun_control_other_|','income_tax_other_|','immigration_other_|']
interestedEnts = ['gun_control_other_|']
for e in interestedEnts:
    rows.append(ent_ind[e])
rows = np.array(rows)
#rows = np.array([10,33,34,40])
XEn = _XEn[rows,:]
XEt = _XEt[rows,:]
NEb = _NEb[:,rows]
TEb = _TEb[:,rows]
#####

XEn = _XEn
XEt = _XEt
NEb = _NEb
TEb = _TEb
"""
print len(tweetsObjDedup),len(tweetsIn),len(set(tweetsIn))
t0 = time()

newsScore = XN.dot(Pw_zs[0][:,i])

tweetsScore = XT.dot(Pw_zs[0][:,i])
#tweetsScore = []
#for t in tweetsInObj:
#    tweetsScore.append(float(t.pop))
#tweetsScore = np.array(tweetsScore)

print( "init score in "+str(time() - t0))
t0 = time()

NE_ = XN.dot(XEn.T) #.multiply(NEb)
TE_ = XT.dot(XEt.T) #.multiply(TEb)

##### weight by pop
TE_indptr = TE_.indptr
TE_data = TE_.data
for (XTid,t) in enumerate(tweetsInObj):
    st = TE_indptr[XTid]
    en = TE_indptr[XTid+1]
    TE_data[st:en]*=float(t.pop)
#####

NE,EN = normBypartite(NE_)
TE,ET = normBypartite(TE_)
print( "graph constr in "+str(time() - t0))
t0 = time()

nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 1, 1, 5)
print( "trihits in "+str(time() - t0))
t0 = time()


16701 16700 16700
init score in 0.00196981430054
graph constr in 0.241264820099
trihits in 0.0306899547577


In [182]:
nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, 1,1, 5)
#nScore, tScore = triHits(np.ones(newsScore.shape), np.ones(tweetsScore.shape), NE, EN, TE, ET, 1, 1, 50)
print np.linalg.norm(norm2(newsScore)[:10] - nScore[:10])
print "========="
kSummary=20
printSummary(norm2(newsScore),norm2(tweetsScore),sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
print "*****"
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)

for k in range(len(tScore)):
    tScore[k] /= float(tweetsInObj[k].pop)
print "*****"
print "*****"
print "*****"
print "*****"
print "*****"
print "*****"
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)    

0.0467887253181
342 0.129141109921 2016-01-13 04:59:03 hillary clinton defends  chelsea clinton's attacks against  bernie sanders | Hillary Clinton Defends Chelsea Clinton's Attacks Against Bernie Sanders
503 0.111838697061 2016-01-12 09:08:00 bernie sanders takes slim lead over  hillary clinton in new  iowa poll | Bernie Sanders Takes Slim Lead Over Hillary Clinton in New Iowa Poll
292 0.111136825576 2016-01-12 15:32:36 hillary clinton's lead over  bernie sanders slipping in new poll | Hillary Clinton's Lead Over Bernie Sanders Slipping in New Poll
376 0.108724180267 2016-01-13 10:21:25 what happens if  hillary clinton loses to  bernie sanders in  iowa? | What happens if Hillary Clinton loses Iowa?
548 0.106436692656 2016-01-08 15:01:43 bernie sanders is ahead of  hillary clinton by a 50-37 percent margin among new  hampshire  democratic primary voters. | Fox News Poll: Sanders up by 13 points in New Hampshire
415 0.100592234702 2016-01-10 08:19:00 many  republicans have argued they a

In [174]:
printTopWords(59,TE_,ents,20)
printTopWords(59,TE,ents,20)
#printTopWords(468,NE,ents,20)
i

[(u'bernie_sanders_per_|', 1002.2700915899359), (u'single-payer_health_care_other_|', 770.6574215542721), (u'chelsea_clinton_per_|', 459.22779351233959), (u'hillary_rodham_clinton_per_|', 382.2703416217999), (u'bill_clinton_per_|', 311.92962335841776), (u'caucus_other_|', 284.44473046859264), (u'vermont_loc_|', 271.05769622484041), (u'united_states_senate_org_|', 241.56919754843511), (u'gun_control_other_|', 229.79274111106844), (u'medicare_(united_states)_other_|', 229.38529208189544), (u'iowa_loc_|', 225.93914275449558), (u'democratic_party_(united_states)_org_|', 203.57688239098252), (u"martin_o'malley_per_|", 198.75825507475597), (u'united_states_presidential_election_in_iowa,_2008_other_|', 187.6781644164839), (u'minimum_wage_other_|', 183.22089688250696), (u'central_america_other_|', 163.2212537544458), (u'fee_other_|', 142.7353257854794), (u'donald_trump_per_|', 123.83384856441549), (u'united_states_presidential_election,_2008_other_|', 118.04304166784799), (u'trail_other_|', 11

7

In [135]:
from utils import loadPickle,triHits,norm2,printTopWords,coHits
entsScore = XEn.dot(Pw_zs[0][:,i])
#printRankedEnt(entsScore,ents,100,DISPLAY=True)
nScore, eScore = coHits(newsScore, entsScore, NE, EN, 1, 1, 20)
printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,20)

488 0.0685041480181 2016-01-08 16:44:46 the sanders campaign said that as a  senate candidate in  new york in 2000, mrs.  clinton favored licensing all gun buyers and establishing a  registry of all  handgun sales, but that, running for president in 2008, she said she no longer believed such "blanket rules" made sense at the  federal level. | Obama's Pledge on Gun Control Sets Off Skirmish Between Hillary Clinton and Bernie Sanders
418 0.0663905688025 2016-01-10 08:19:00 clinton also used the sunday morning interview, where she appeared live from a  fire station in  chappaqua, new york, to knock sanders on guns, something her campaign has been eager to do since  president  barack obama indicated on thursday that he would not campaign for any candidate who does not support common-sense gun reform. | Clinton responds to Trump's personal attacks
139 0.063385546538 2016-01-12 02:00:34 clinton advisers pointed to the  nbc poll as evidence of passion for mrs.  clinton, while sanders advisers

In [183]:
res1 = printRankedEnt(NE.T.dot(nScore),ents,100)
print "===="
res2 = printRankedEnt(TE.T.dot(tScore),ents,100)
print "==="
for e in ents:
    print res2[e] - res1[e],e
"""
for e in ents:
    print entscore[e],e
print TE.getrow(1).toarray()    

print NE.getrow(80).toarray()
print NE.getrow(88).toarray()
print NE.getrow(99).toarray()
print NE.getrow(65).toarray()
"""    

====
===
0 hillary_rodham_clinton_per_|
0 bernie_sanders_per_|
-2 democratic_party_(united_states)_org_|
0 bill_clinton_per_|
1 iowa_loc_|
-4 new_hampshire_primary_other_|
-8 vermont_loc_|
-6 republican_party_(united_states)_org_|
1 united_states_senate_org_|
-3 united_states_presidential_election,_2008_other_|
-18 income_tax_other_|
0 legislation_other_|
2 barack_obama_per_|
2 chelsea_clinton_per_|
1 general_election_other_|
1 new_hampshire_loc_|
5 fox_news_channel_org_|
5 lawsuit_other_|
-2 gun_control_other_|
3 legal_liability_other_|
3 trail_other_|
10 deportation_other_|
-9 modern_liberalism_in_the_united_states_other_|
-17 cedar_rapids,_iowa_loc_|
11 single-payer_health_care_other_|
11 medicare_(united_states)_other_|
2 central_america_other_|
14 caucus_other_|
-5 fee_other_|
8 donald_trump_per_|
7 narrative_other_|
4 united_states_presidential_election_in_iowa,_2008_other_|
0 political_party_other_|
-7 immigration_other_|
11 minimum_wage_other_|
-4 patient_protection_and_afforda

'\nfor e in ents:\n    print entscore[e],e\nprint TE.getrow(1).toarray()    \n\nprint NE.getrow(80).toarray()\nprint NE.getrow(88).toarray()\nprint NE.getrow(99).toarray()\nprint NE.getrow(65).toarray()\n'

In [154]:
from utils import printTopWords
for k in range(topK):
#i=18
    print ents[k]
    printTopWords(i,XEn,vocab,20)
    print "+++++++++"

hillary_rodham_clinton_per_|
[(u'republican', 0.75696241334363046), (u'republicans', 0.3915322827639468), (u'gop', 0.20630838461603315), (u'primary', 0.2045851019532649), (u'voters', 0.16597533882744858), (u'field', 0.14093796023121016), (u'candidates', 0.11789050549487609), (u'donald', 0.09928302947259697), (u'front-runner', 0.078306456552789352), (u'governors', 0.070468980115605082), (u'blasts', 0.063728031736693266), (u'contest', 0.063728031736693266), (u'trump', 0.061020865500538826), (u'say', 0.058945252747438046), (u'leading', 0.052204304368526237), (u'ad', 0.052204304368526237), (u'white', 0.052204304368526237), (u'race', 0.049641514736298485), (u'including', 0.049641514736298485), (u'president', 0.04742152537927103)]
+++++++++
bernie_sanders_per_|
[(u'republican', 0.75696241334363046), (u'republicans', 0.3915322827639468), (u'gop', 0.20630838461603315), (u'primary', 0.2045851019532649), (u'voters', 0.16597533882744858), (u'field', 0.14093796023121016), (u'candidates', 0.1178905